In [23]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [15]:
data = TextLoader("books/a_message_to_garcia.txt", encoding="utf-8").load()
text_splitter = RecursiveCharacterTextSplitter(separators="\n", chunk_size=1000, chunk_overlap=200)
raw_documents = text_splitter.split_documents(data)

In [18]:
raw_documents[0].page_content

"A MESSAGE TO GARCIA\n\nBeing a Preachment\n\nby\n\nElbert Hubbard.\n\n\n\n\n\n\n\n[Illustration: Elbert Hubbard]\n\n\n\n[Illustration]\n\n\n\nDone into a Printed Book\nby the Roycrofters at\nTheir Shop, Which Is in East\nAurora, Erie County, N.Y.\nCopyright 1914 by Elbert Hubbard\n\n\n\n\n\nAPOLOGIA\n\n\n\n\nHORSE SENSE\n\nIf you work for a man, in Heaven's name work for him. If he pays wages\nthat supply you your bread and butter, work for him, speak well of\nhim, think well of him, and stand by him, and stand by the institution\nhe represents. I think if I worked for a man, I would work for him.\nI would not work for him a part of his time, but all of his time. I\nwould give an undivided service or none. If put to the pinch, an\nounce of loyalty is worth a pound of cleverness. If you must vilify,\ncondemn, and eternally disparage, why, resign your position, and when\nyou are outside, damn to your heart's content. But, I pray you, so\nlong as you are a part of an institution, do not 

In [25]:
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-exp-03-07")